In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
  !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
  !pip install sentencepiece protobuf huggingface_hub hf_transfer
  !pip install transformers==4.51.3
  !pip install --no-deps unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True


fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/codellama-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.6.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
!pip install -U datasets
from datasets import load_dataset
dataset = load_dataset("CyberNative/Code_Vulnerability_Security_DPO", split="train")
print(dataset.column_names)

README.md:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

secure_programming_dpo.json:   0%|          | 0.00/6.87M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4656 [00:00<?, ? examples/s]

['lang', 'vulnerability', 'system', 'question', 'chosen', 'rejected']


In [5]:
from unsloth import to_sharegpt, standardize_sharegpt, apply_chat_template

# Step 1: Merge columns into prompt/response format
dataset = to_sharegpt(
    dataset,
    merged_prompt=(
        "{question}\n\n"
        "The following response was rejected due to a security vulnerability:\n"
        "{rejected}\n\n"
        "Reason: {vulnerability}\n\n"
        "Please provide a better and secure alternative."
    ),
    output_column_name="chosen",
    conversation_extension=3
)

# Step 2: Convert to 'INPUT' and 'OUTPUT' columns
dataset = standardize_sharegpt(dataset)

# Step 3: Filter out invalid rows

# ✅ Step 4: Use properly formatted chat_template
chat_template = """USER:
{INPUT}

ASSISTANT:
{OUTPUT}

USER:
{INPUT}

ASSISTANT:
{OUTPUT}

"""

# Step 5: Apply the chat template
dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
    default_system_message="You are a helpful assistant that fixes vulnerable code."
)


Merging columns:   0%|          | 0/4656 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/4656 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/4656 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/4656 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/4656 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/4656 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/4656 [00:00<?, ? examples/s]

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/4656 [00:00<?, ? examples/s]

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/4656 [00:00<?, ? examples/s]

In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,656 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 39,976,960/7,000,000,000 (0.57% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.923500
2,0.940300
3,0.844200
4,0.896300
5,0.828000
6,0.806200
7,0.734400
8,0.795100
9,0.737500
10,0.765200


In [8]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

FastLanguageModel.for_inference(model)

messages = [
    {
        "role": "user",
        "content": (
            "Write a function to hash a password.\n\n"
            "The following response was rejected due to a security vulnerability:\n"
            "```python\n"
            "def hash_password(password):\n"
            "    return hashlib.md5(password.encode()).hexdigest()\n"
            "```\n\n"
            "Reason: Use of MD5 which is cryptographically broken and not recommended for password hashing.\n\n"
            "Please provide a better and secure alternative."
        )
    }
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt=True)

_ = model.generate(
    input_ids,
    streamer=text_streamer,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id,
)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


```python
import hashlib

def hash_password(password):
   salt = b'salt'
   return hashlib.pbkdf2_hmac('sha256', password.encode(), salt, 100000)
```

</s>


In [9]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [10]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [11]:
%cd /content/
!git clone --recursive https://github.com/ggerganov/llama.cpp
%cd llama.cpp
!make clean && make -j


/content
Cloning into 'llama.cpp'...
remote: Enumerating objects: 52891, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 52891 (delta 34), reused 16 (delta 16), pack-reused 52825 (from 4)
Receiving objects: 100% (52891/52891), 128.28 MiB | 26.68 MiB/s, done.
Resolving deltas: 100% (38358/38358), done.
Submodule 'kompute' (https://github.com/nomic-ai/kompute.git) registered for path 'ggml/src/ggml-kompute/kompute'
Cloning into '/content/llama.cpp/ggml/src/ggml-kompute/kompute'...
remote: Enumerating objects: 9122, done.        
remote: Counting objects: 100% (155/155), done.        
remote: Compressing objects: 100% (69/69), done.        
remote: Total 9122 (delta 109), reused 86 (delta 86), pack-reused 8967 (from 3)        
Receiving objects: 100% (9122/9122), 17.59 MiB | 35.38 MiB/s, done.
Resolving deltas: 100% (5728/5728), done.
Submodule path 'ggml/src/ggml-kompute/kompute': checked out '4565194ed7c32d1d2efa32ceab4d

In [12]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/codellama-7b-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# Load LoRA adapter
model.load_adapter("/content/lora_model")

# Merge and save
model.save_pretrained("/content/merged_model")
tokenizer.save_pretrained("/content/merged_model")

from unsloth.save import save_to_gguf

save_to_gguf(
    model_type = "llama3",
    model_dtype = "q8_0",  # or q4_k_m, etc.
    is_sentencepiece = True,
    model_directory = "/content/merged_model",
    quantization_method = "llama.cpp",
    first_conversion = True
)


if True: model.save_pretrained_gguf("model", tokenizer,)

==((====))==  Unsloth 2025.6.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

AssertionError: 

In [ ]:
import subprocess

subprocess.Popen(["ollama", "serve"])
import time

time.sleep(3)

In [ ]:
print(tokenizer._ollama_modelfile)
!ollama create unsloth_model -f ./model/Modelfile

In [ ]:
from google.colab import files


!zip -r lora_model.zip lora_model
files.download("lora_model.zip")